In [11]:
import tensorflow
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import GlobalMaxPooling2D

In [12]:
model=ResNet50(weights='imagenet',include_top=False, input_shape=(224,224,3))
# include_top=False means we are avoiding the input layer
model.trainable=False

In [8]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [13]:
model=tensorflow.keras.Sequential([
    model, 
    GlobalMaxPooling2D()  
    # Without max pooling layer o/p shape will be 7,7,2048, which is not a single vector. Using just MaxPooling2D will make it 3,3,2048
    # GlobalMaxPooling2D will give 
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


In [1]:
import cv2
import numpy as np
from numpy.linalg import norm

In [5]:
img=cv2.imread("./uploads/1636.jpg")
img=cv2.resize(img, (224,224))

In [6]:
img=np.array(img)

In [7]:
img.shape

(224, 224, 3)

In [8]:
# (number_of_images, 224, 224, 3)
expand_img=np.expand_dims(img, axis=0)

In [9]:
expand_img.shape

(1, 224, 224, 3)

In [14]:
pre_img=preprocess_input(expand_img)

In [15]:
pre_img.shape

(1, 224, 224, 3)

In [16]:
result=model.predict(pre_img)
result.shape

# 1 means, a single image and 2048 is the feature extracted from the image

(1, 2048)

In [31]:
result=model.predict(pre_img).flatten()

In [37]:
normalized=result/norm(result)

In [38]:
normalized.shape

(2048,)

In [39]:
def extract_feature(img_path, model):
    img=cv2.imread(img_path)
    img=cv2.resize(img, (224,224))
    img=np.array(img)
    expand_img=np.expand_dims(img, axis=0)
    pre_img=preprocess_input(expand_img)
    result=model.predict(pre_img).flatten()
    normalized=result/norm(result)
    return normalized

In [40]:
extract_feature("1636.jpg",model)

array([0.00596654, 0.00037372, 0.00811542, ..., 0.0147143 , 0.01509577,
       0.02696898], dtype=float32)

In [46]:
import os
from tqdm import tqdm

In [42]:
filename=[]
feature_list=[]

for file in os.listdir('Dataset'):
    filename.append(os.path.join('Dataset',file))

In [44]:
filename[0:5]

['Dataset\\10037.jpg',
 'Dataset\\10039.jpg',
 'Dataset\\10054.jpg',
 'Dataset\\10096.jpg',
 'Dataset\\10097.jpg']

In [47]:
for file in tqdm(filename):
    feature_list.append(extract_feature(file,model))

100%|█████████████████████████████████████████████████████████████████████████████| 2906/2906 [14:08<00:00,  3.43it/s]


In [48]:
import pickle

pickle.dump(feature_list,open('featurevector.pkl','wb'))
pickle.dump(filename,open('filenames.pkl','wb'))
